## Lab 4 - Sentiment Analysis

Your task is to determine the tonality of the text on a scale of 1 (negative) to 10 (positive).
Input: texts separated by newlines (\ n).
Conclusion: for each text from the input, derive a tonal score from 1 to 10. The separator between the conclusions for different texts is the line feed (\ n).

For training, you can use a collection of texts and their corresponding grades. The training collection file corresponds to the input format, the grades file corresponds to the output format; encoding - UTF-8.

The square root of the root mean square error (Root Mean Squared Error) is used as an estimate.

You can briefly describe the solution in Read.me file at lab folder. The information will be useful to the instructor of the course in order to gain an idea of the methods and approaches used. Notebook must contain classification report, namely the comparison between the output of classificator and target var.
Sample Input:

По сравнению с предыдущими произведениями гораздо слабее. Хотя динамика событий осталась, но сама идея и ее воплощение...... Очень жаль.
Это просто СУПЕР... Давно ничего подобного не читала.
Sample Output:

2

8

In [1]:
import torch
import numpy as np
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel, BertConfig, BertPreTrainedModel, BertForPreTraining, BertForMaskedLM
import torch.nn as nn
from tqdm import tqdm, tqdm_notebook
import os
import re
RUBERT_PATH = 'ru_conversational_cased_L-12_H-768_A-12_pt'
modelpath = os.path.join(RUBERT_PATH,'pytorch_model.bin')

In [2]:
os.path.isfile(os.path.join(RUBERT_PATH,'pytorch_model.bin'))

True

In [3]:
tokenizer = BertTokenizer.from_pretrained(RUBERT_PATH, do_lower_case=False)
config = BertConfig.from_json_file(os.path.join(RUBERT_PATH,'bert_config.json'))
model = BertForPreTraining.from_pretrained(modelpath, config=config)
model.eval()
from torch import load
di = load(modelpath)

In [4]:
def mean_calc(text): 
    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    segments_ids[0] = 0
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    predictions = model(tokens_tensor, token_type_ids=segments_tensors)
    array_of_thensors = []
    for i in range(len(tokenized_text)):
        array_of_thensors.append((predictions[0][0][i].detach().numpy()))
       
    mean = np.mean(array_of_thensors, axis = 0)
    return mean

In [19]:
df = pd.DataFrame()
df = pd.read_csv('texts_train.txt', sep="\t", encoding='UTF-8',header=0)
df['score'] = pd.read_csv('scores_train.txt', sep="\t", encoding='UTF-8',header=0)
df.columns = ['text', 'score']
df.head()

,text,score
0,"думал, что будет лучше идея очень интересна - ...",7
1,с творчеством Головачева я познакомился посред...,10
2,"то-то я и в большое неудовольствие прочитал ""А...",5
3,как мне показалось местами сильно смахивает на...,6
4,от первой части книги просто оторваться не мог...,9


In [15]:
unique_grades = np.sort(df['score'].unique())
print('Original range of scores: ',unique_grades)
unique_grades  = [1,5,10]
print('New range of scores: ',unique_grades)

Original range of scores:  [ 1  2  3  4  5  6  7  8  9 10]
New range of scores:  [1, 5, 10]


In [8]:
df_new = pd.DataFrame()
comment = []
grade = []

for grades in unique_grades:
    counter = 0
    for i in range(len(df)):
    #for i in range(000/(len(unique_grades))):
        if (counter<900):
            if (grades == df['score'][i]):
                comment.append(df['text'][i])
                grade.append(df['score'][i])
                counter = counter + 1 
df_new['text'] = comment
# lower the range of classifiers to ranging 1-3, where 1 is negative and 3 positive
df_new['score'] = grade
df_new['score'] = df_new['score'].replace(5, 2)
df_new['score'] = df_new['score'].replace(10, 3)
print('substituted range of scores: ',np.sort(df_new['score'].unique()))
df_new.head()

substituted range of scores:  [1 2 3]


,text,score
0,Хотелось долго плеваться после того как я с тр...,1
1,"ИМХО, в серии о Свароге Бушков исписался. Если...",1
2,Много был наслышан об этом авторе и вот взялся...,1
3,"Да, тяжело читать безграмотную фэнтези. Хотя, ...",1
4,Попытка свести сюжет предыдущих книг в единое ...,1


In [9]:
array_of_mean_tensors = []
grade = []
comment = []
errors = 0
for i in range(len(df_new)):
    try: 
        array_of_mean_tensors.append(mean_calc(df_new['text'][i]))
        grade.append(df_new['score'][i])
        comment.append(df_new['text'][i])
    except:
        #in some cases, from my point of view, when comment is too large, the data has not processed, and it cause some errors
        errors = errors + 1
        #print('Too long or to short comment')

In [10]:
df_with_tensors = pd.DataFrame()
df_with_tensors['text'] = comment
df_with_tensors['tensor'] = array_of_mean_tensors
df_with_tensors['score'] = grade
df_with_tensors.to_csv('df_with_thensors_900.csv', sep = '|')

In [11]:
X = array_of_mean_tensors
y = df_with_tensors['score']

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=42)

In [13]:
from sklearn.metrics import accuracy_score as ac
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error as mse
def metrics(y_test, y_pred, name_of_alg): 
    mae = np.mean(abs(y_test - y_pred))
    rmse = (mse(y_test, y_pred))**0.5
    print('\bMetrics and score for ', name_of_alg,':')
    print('Accuracy score from rmse:', 100 - np.mean(100 * (rmse / y_test)))
    print('Accuracy score from mae:', 100 - np.mean(100 * (mae / y_test)))
    print('Classification_report: ')
    print(classification_report(y_test, y_pred))
    print('Confusion matrix')
    print(confusion_matrix(y_test, y_pred))
    print('Root MSE')    
    print(rmse)
    print('Root MSE')    
    print(mae)

In [14]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100)
clf = clf.fit(X_train, y_train)

In [16]:
y_pred = clf.predict(X_test)
print(metrics(y_test,y_pred, 'Random forest'))

Metrics and score for  Random forest :
Accuracy score from rmse: 66.6690043047625
Accuracy score from mae: 82.25454552071518
Classification_report: 
              precision    recall  f1-score   support

           1       0.65      0.16      0.26        69
           2       0.65      0.77      0.70       177
           3       0.74      0.81      0.78       182

    accuracy                           0.69       428
   macro avg       0.68      0.58      0.58       428
weighted avg       0.69      0.69      0.66       428

Confusion matrix
[[ 11  42  16]
 [  5 137  35]
 [  1  33 148]]
Root MSE
0.6538884411411047
Root MSE
0.34813084112149534
None


In [18]:
for i in range(len(y_test)):
    index = y_test.index[i]
    print(df_with_tensors['text'][index])
    print('Target: ', str(y[index]))
    print('Predicted: ', str(y_pred[i]))
    print('-'*100)

я умерла и воскресла!... Вечная Книга! не только для чтения, но и для понимания - смыслов нереально много
Target:  3
Predicted:  3
----------------------------------------------------------------------------------------------------
Почему никто никогда не говорил, что "Страна чудес без тормозов и Конец света" Мураками - это настоящий киберпанк?! Все признаки налицо: хакеры, мегакорпорации, всякие компьютерные и цифровые штуки с интеллектом, маргинальный образ жизни главного персонажа... Кайф! Только что добила эту книгу - я под впечатлением... Очень необычно, как и все у Мураками. До сих пор я была убеждена, что "Охота на овец" - это лучшее, что он написал. Открою свой маленький литературный секрет: книги Харуки читаю медленно и порционно - не больше одной в год. Иначе, если этого сумасшедшего японца читать залпом, в голове остается мешанина, а не странное звенящее послевкусие. Этот год под знаком "Страны чудес...". В общем, слов нет, одни эмоции.
Target:  3
Predicted:  2
-------------

Target:  3
Predicted:  3
----------------------------------------------------------------------------------------------------
Всем, кто собирается читать эту книгу (те, кто уже начал, сами всё поняли :)))! Берегитесь! теперь ваши руки постоянно будут заняты очередной книгой Оксаны Панкеевой! вы не сможете спать по ночам, потому что бросить книгу на середине у вас не хватит силы воли! А дочитав одну, сразу потянетесь за следующей! (мда, алкоголизм, последняя стадия...) Когда вы дочитаете 5-ую книгу, у вас начнется ломка, и вы каждый день будете лазать на сайт www.pankeewa.org.ru, чтобы узнать, не вышла ли наконец, шестая книга :) Короче, вы станете бессменным фанатом Панкеевой и славного королевства Ортан!
Target:  3
Predicted:  3
----------------------------------------------------------------------------------------------------
Классная книга! Наверно, лучшая - или, как минимум, одна из лучших у Перумова. Только вот частые и длительные описания батльных сцен несколько надоедают.
Targe